In [81]:
import matplotlib.pyplot as plt #plotting
import numpy as np
import os #to access directory
from tqdm import tqdm #counting files

import seaborn as sns #visual beautification
import cv2
import io #input/ouput from local
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle #shuffles matrics randomly with continuation of same pattern
from tensorflow.keras.preprocessing.image import ImageDataGenerator #augmentation, annotation
from tensorflow.keras.applications import EfficientNetB7 #B0-B7

#EarlyStopping: to avoid overfitting
#ReduceLROnplateau: Reduce Learning Rate based on weight
#TensorBoard: Visualization toolkit of tensorflow
#ModelCheckpoint: save the model or weight at every epoch for improve performance and avoid repeat

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import to_categorical #encoding

#result checking
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from warnings import filterwarnings #ignore deprecation
from PIL import Image #pillow for image open, rotate and display

import ipywidgets as widgets  #for button
from IPython.display import display, clear_output 

**To see all files one by one and findout unnecessary/hidden/corrupted file, name mismatch**

In [ ]:
#os.listddir = to show all directory
#os.walk = to show each files with dir
#os.path.join=merge the paths
for dir_name, _, filenames in os.walk("../input/alzheimersdisease5classdatasetadni"):
    for filename in filenames:
        print(os.path.join(dir_name, filename))
    

In [83]:
#labels based on classwise folder name in directory
labels=["Final AD JPEG", "Final CN JPEG", "Final EMCI JPEG", "Final LMCI JPEG", "Final MCI JPEG"]

**Prepare train test Data Together**

In [115]:
X_train=[]
y_train=[]

image_size=162
for i in labels:
    folderDirectory = os.path.join("../input/alzheimersdisease5classdatasetadni/Alzheimers-ADNI", "train", i) #tagging labels by i
    for j in tqdm(os.listdir(folderDirectory)): #tqdm counting
        image=cv2.imread(os.path.join(folderDirectory, j)) #image to array
        image=cv2.resize(image, (image_size, image_size))
        
        X_train.append(image)
        y_train.append(i)
for i in labels:
    folderDirectory=os.path.join("../input/alzheimersdisease5classdatasetadni/Alzheimers-ADNI","test", i)
    for j in tqdm(os.listdir(folderDirectory)):
        image=cv2.imread(os.path.join(folderDirectory, j))
        image=cv2.resize(image, (image_size,image_size))
        
        X_train.append(image)
        y_train.append(i)
#now we need to convert it to numpy array to make it workable
X_train=np.array(X_train)
y_train = np.array(y_train)
   
    

In [116]:
print("X_train Shape:", X_train.shape)
print("y_train Shape:", y_train.shape)


**Count unique value for each class**

In [117]:
_,y_trainImageInfo= np.unique(y_train, return_counts =True)
print("Final AD JPEG:")
print("counts:", y_trainImageInfo[0])
print("Final CN JPEG:")
print("counts:", y_trainImageInfo[1])
print("Final EMCI JPEG:")
print("counts:", y_trainImageInfo[2])
print("Final LMCI JPEG:")
print("counts:", y_trainImageInfo[3])
print("Final MCI JPEG:")
print("counts:", y_trainImageInfo[4])

In [118]:
#shuffle based on randomState
X_train, y_train = shuffle(X_train, y_train, random_state=101)

In [119]:
print("Shape of X_train:",'\n',X_train.shape)
print("Shape of y_train:",'\n',y_train.shape)

In [120]:
print(y_train[0])
print(y_train[100])
print(y_train[200])
print(y_train[500])
print(y_train[750])
print(y_train[1000])
print(y_train[1295])

**Train and Test Split**

In [121]:
X_train, X_test, y_train, y_test =train_test_split(X_train, y_train, test_size=0.10, random_state=101)

In [122]:
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

In [123]:
print("y_train Index 0:", y_train[0])
print("y_test Index 0:", y_test[0])

**Categorical/label/one hot encoding**

In [124]:
y_train_new=[]
y_test_new=[]

for i in y_train:
    y_train_new.append(labels.index(i)) #indexing value of y_train
y_train = y_train_new
y_train = to_categorical(y_train)

for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = to_categorical(y_test)

In [125]:
print("shape of y_train", y_train.shape)
print("array of y_test", y_test.shape)
print("array of y_train", y_train)




**max value in array for respective image class**

In [126]:
#maxma value of that array of respective image
print(np.argmax(y_train[0]))
print(np.argmax(y_train[200]))
print(np.argmax(y_train[450]))

In [127]:
efficient = EfficientNetB7(weights ="imagenet", include_top=False, input_shape=(image_size, image_size, 3))


In [128]:
import tensorflow as tf

In [130]:
#pooling can be avg or max
#dropout range 0.3 -0.5 which means 30-50% neurons will be dropped to avoid overfitting
model=efficient.output
model=tf.keras.layers.GlobalMaxPooling2D()(model)
model=tf.keras.layers.Dropout(0.5)(model)
model=tf.keras.layers.Dense(5, activation="softmax")(model)
model=tf.keras.models.Model(inputs=efficient.input, outputs=model)

In [131]:
model.summary()

In [132]:
#loss for categorical value, adam perform best
model.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics =["accuracy"])

In [133]:
#.h5 = Hierarchical Data Format Ver. 5 file, verbose =1, to see execution
tensorboard= TensorBoard(log_dir="logs")
checkpoint= ModelCheckpoint("efficient.h5",
                           monitor="val_accuracy", verbose=1,
                           mode="auto", save_best_only=True)
#monitor: quantity to be monitored.     
#factor: factor by which the learning rate will be reduced. 
#patience: number of epochs with no improvement after which learning rate will be reduced.     
#verbose: int. 0: quiet, 1: update messages.
#min_delta: early stopping of epochs
reduce_lr=ReduceLROnPlateau(monitor="val_accuracy",
                           factor=0.5,
                           patience=3,verbose=1,
                           mode="auto", min_delta=0.001)

                               

**Fit the model**

In [134]:
history = model.fit(X_train, y_train,
                   validation_split=0.1,
                   epochs=15, verbose=1,
                   batch_size=32,
                   callbacks =[tensorboard, checkpoint, reduce_lr])

In [135]:
def accuracy_loss_plot (history):
    fig = plt.figure(figsize=(10,10))
    
    plt.subplot(221)
    plt.plot(history.history["accuracy"], 'bo--', label="accuracy")
    plt.plot(history.history['val_accuracy'], 'ro--', label="val_accuracy")
    plt.title("Training Data Accuracy Measurements")
    plt.xlabel("Number of epochs")
    plt.ylabel("Accuracy")
    plt.grid()
    plt.legend()
    plt.tight_layout()
    
    plt.subplot(222)
    plt.plot(history.history["loss"], "bo--", label="loss")
    plt.plot(history.history["val_loss"], "ro--", label = "val_loss")
    plt.title("Training Data Loss")
    plt.xlabel("Number of epochs")
    plt.ylabel("loss")
    plt.grid()
    plt.legend()
    plt.tight_layout()
    
    plt.show()

In [136]:
accuracy_loss_plot(history)

In [137]:
X_testloss=model.evaluate(X_test, y_test, verbose=1)

In [138]:
y_pred = model.predict(X_test)
y_pred=np.argmax(y_pred, axis=1)
y_test_new = np.argmax(y_test, axis=1)

In [139]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
print(classification_report(y_test_new, y_pred))

In [140]:
cm=confusion_matrix(y_test_new, y_pred)
fig, ax = plt.subplots(figsize=(20,5)) 
sns.heatmap(cm, annot=True, linewidths=2, ax=ax)

In [141]:
display_c_m = ConfusionMatrixDisplay(cm, display_labels=labels)
fig, ax = plt.subplots(figsize=(25,10)) 
display_c_m.plot(cmap='mako',ax=ax,xticks_rotation=60)
plt.title("Confusion Metrics")
plt.show()

In [142]:
def imagePrediction(upload):
    for name, fileinfo  in uploader.value.items():
        image = Image.open(io.BytesIO(fileinfo['content'])) #Image for PIL and io=input/output locall
    
    images = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    images = cv2.resize(images,(162, 162))
    images = images.reshape(1, 162, 162, 3)
    prd = model.predict(images)
    prd = np.argmax(prd, axis = 1)[0]
    
    if prd==0:
        prd="Final AD JPEG"
    elif prd==1:
        prd="Final CN JPEG"
    elif prd==2:
        prd="Final EMCI JPEG"
    elif prd==3:
        prd="Final LMCI JPEG"
    elif prd==4:
        prd="Final MCI JPEG"
    
    if prd!=1:
        print(f"Model Predict That is a {prd}")
        

**To upload files/images**

In [143]:
uploader = widgets.FileUpload()
display(uploader)

**Image Predition for Alzheimers**

In [112]:
button=widgets.Button(description="Predict")
out=widgets.Output()

def on_button_click(_): 
    with out:
        clear_output()
        try: 
            imagePrediction(uploader)
        except:
            print("Enter the correct image file")
button.on_click(on_button_click)
widgets.VBox([button, out])
            